In [1]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop

from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow import keras


from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
d_list = list(test_dataset)

X_train, Y_train = get_X_Y(train_dataset)

X_test, Y_test = get_X_Y(test_dataset)

#X = []
#for i in range(0, len(Y_train) - 1):
#    X.append([Y_train[i], X_train[i]])




In [3]:
from sklearn.datasets import make_regression
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot
from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
X_train, Y_train = get_X_Y_vectorized_int(train_dataset)
X_test, Y_test = get_X_Y_vectorized_int(test_dataset)


model = Sequential()
#model.add(Flatten(input_shape=(len(X_train), 17)))
model.add(Dense(17))
model.add(Dense(128, activation='relu'))
model.add(Dense(42))

model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(np.array(X_train), np.array(Y_train), epochs=10)
model.summary()
#print(Y_train)

Epoch 1/10
7875/7875 [==============================] - 4s 482us/step - loss: 0.0186
Epoch 2/10
7875/7875 [==============================] - 4s 481us/step - loss: 0.0145
Epoch 3/10
7875/7875 [==============================] - 4s 472us/step - loss: 0.0141
Epoch 4/10
7875/7875 [==============================] - 4s 488us/step - loss: 0.0139
Epoch 5/10
7875/7875 [==============================] - 4s 479us/step - loss: 0.0138
Epoch 6/10
7875/7875 [==============================] - 4s 486us/step - loss: 0.0137
Epoch 7/10
7875/7875 [==============================] - 4s 491us/step - loss: 0.0137
Epoch 8/10
7875/7875 [==============================] - 4s 498us/step - loss: 0.0136
Epoch 9/10
7875/7875 [==============================] - 4s 492us/step - loss: 0.0136
Epoch 10/10
7875/7875 [==============================] - 4s 484us/step - loss: 0.0135
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param 

In [5]:
test_acc = model.evaluate(np.array(X_test), np.array(Y_test), verbose=2)

print('\nTest loss:', test_acc)

3113/3113 - 1s - loss: 0.0129

Test loss: 0.012883305549621582


In [14]:
def get_prediction_list(model):
    """
    The model is like "model = LinearRegression()" (or other algorithms)
    """

    x_pred, info = get_pics_from_file("../tohack/pics_LOGINMDP.bin")

    res = []

    poor_progress_bar = 0
    for i in range(len(x_pred) // 400 + 1):
        print('#', end="")
    print("")
    for trame_pred in x_pred:
        if poor_progress_bar % 400 == 0:
            print('#', end="")
        poor_progress_bar += 1
        t = trame_pred
        print(t)
        res = model.predict(t)
        prediction = d_list[res.index(max(res))]
        if prediction != 'NOKEY':
            res.append(prediction)

    print("")
    return res


def get_creds(prediction_list):
    for i_k in range(len(prediction_list)):
        if prediction_list[i_k] == 'CTRL' and i_k + 2 < len(prediction_list) and prediction_list[i_k + 2] == 'SUPPR':
            print('#####################')
            print(prediction_list[i_k:i_k + 42])


In [15]:
res = get_prediction_list(model)
print(res)

############################
#[0.47546387 0.42724609 0.17089844 0.3729248  0.51818848 0.6060791
 0.65124512 0.67260742 0.5255127  0.28869629 0.42602539 0.46875
 0.35644531 0.51574707 0.69702148 0.55297852 0.27099609]


ValueError: in user code:

    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1461 run_step  **
        outputs = mo